In [1]:
import pandas as pd
import glob
import numpy as np
import matplotlib as mpl

if 1:
    mpl.use('pgf')
    mpl.rcParams.update({
        "pgf.texsystem": "pdflatex",
        'font.family': 'serif',
        'text.usetex': True,
        'pgf.rcfonts': False,
    })
from copy import deepcopy
import matplotlib.pyplot as plt


In [2]:
def fix_hist_step_vertical_line_at_end(ax):
    axpolygons = [poly for poly in ax.get_children() if isinstance(poly, mpl.patches.Polygon)]
    for poly in axpolygons:
        poly.set_xy(poly.get_xy()[:-1])

In [3]:
filename_list = glob.glob('/home/guso/Data/combflex2/*.csv')
N = len(filename_list)
nick = filename_list[0].split('/')[-1].split('-')[0]

In [4]:
filter_sim = {
    nick + '-flexible-yes-ismarket-combflex' : 'Combflex 1-0',
    nick + '-flexible-yes-ismarket-combflex_split': 'Combflex S 1-0',
    nick + '-flexible-yes-ismarket-combflex-0.5-0.5': 'Combflex .5-.5',
   nick + '-flexible-yes-ismarket-combflex_split-0.5-0.5': 'Combflex S .5-.5',
    nick + '-long-yes-ismarket-huang': 'Auction H',
    nick + '-long-yes-ismarket-muda': 'Auction M',
    nick + '-short-none-ismarket-p2p': 'P2P',
}

In [5]:
results_cost = {
 'Combflex 1-0': [],
 'Combflex S 1-0': [],
 'Combflex .5-.5': [],
 'Combflex S .5-.5': [],
 'Auction H': [],
 'Auction M': [],
 'P2P': []
}

results_autocons = deepcopy(results_cost)

In [6]:
for filename in filename_list:

    df = pd.read_csv(filename)

    OAC = df[df['1'] == 'Optimal Auto-consumtion'].iloc[0, 3]
    OAG = df[df['1'] == 'Optimal Auto-consumption Guarnatee'].iloc[0, 3]
    ACB = df[df['1'] == 'Auto-consumption Battery'].iloc[0, 3]
    SCD = df[df['1'] == 'Social Cost Default'].iloc[0, 3]
    SCB = df[df['1'] == 'Social Cost Battery'].iloc[0, 3]

    df_ = df.iloc[5:, :].copy()
    df_.columns = ['date', 'param', 'type', 'value']
    #df_ = df_[df_['type'] == 'Total cost']
    df_['relative_cost'] = df_.value / SCB
    df_['relative_autocons'] = df_.value / ACB # with respect to battery

    df_ = df_[df_['param'].isin(filter_sim.keys())]
    
    df_[df_['type'] == 'Total cost'].apply(
        lambda x: results_cost[filter_sim[x.param]].append(x.relative_cost), axis=1)
    
    df_[df_['type'] == 'Auto-consumption'].apply(
        lambda x: results_autocons[filter_sim[x.param]].append(x.relative_autocons), axis=1)
    

In [7]:
n_bins = 50
fig, ax = plt.subplots(figsize=(8, 6))

for k, v in results_cost.items():

    ax.hist(results_cost[k], n_bins, cumulative=True, label=k,histtype='step', linewidth=1.5, density=True)
    fix_hist_step_vertical_line_at_end(ax)
ax.legend()
ax.set_ylabel('Proportion of simulated days')
ax.set_xlabel('Relative Social Cost')
ax.grid(True)
fig.tight_layout()

fig.savefig('/home/guso/Pictures/socialcost_2.pgf')

In [8]:
n_bins = 50
fig, ax = plt.subplots(figsize=(8, 6))

for k, v in results_autocons.items():

    ax.hist(results_autocons[k], n_bins, cumulative=True, label=k,histtype='step', linewidth=1.5, density=True)
    fix_hist_step_vertical_line_at_end(ax)
ax.legend()
ax.set_ylabel('Proportion of simulated days')
ax.set_xlabel('Relative Unmet Demand')
ax.grid(True)
fig.tight_layout()

fig.savefig('/home/guso/Pictures/autocons_2.pgf')